In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("StringIndexer") \
.config("spark.executor.memory","3g") \
.config("spark.driver.memory","10g") \
.getOrCreate()

22/10/13 23:43:19 WARN Utils: Your hostname, batuhan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/10/13 23:43:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/13 23:43:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read \
.option("header","True") \
.option("InferSchema","True") \
.option("sep",",") \
.csv("/home/batuhan/Desktop/datalar/sample_data.csv")

In [4]:
df.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir
0,1,Cemal,35,Isci,Ankara,3500
1,2,Ceyda,42,Memur,Kayseri,4200
2,3,Timur,30,Müzisyen,Istanbul,9000
3,4,Burcu,29,Pazarlamaci,Ankara,4200
4,5,Yasemin,23,Pazarlamaci,Bursa,4800


# VERİ SETİNE ETİKET EKLEME

// Sınıflandırma hedef değişken (etiket-label) yaratmak adına
// Geliri 7000 üstü olanlara ekonomik durum iyi diyelim

In [5]:
from pyspark.sql.functions import *
df1 = df.withColumn("ekonomik durum",
when(col("aylik_gelir") > 7000, "iyi").otherwise("kötü")
)
df1.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik durum
0,1,Cemal,35,Isci,Ankara,3500,kötü
1,2,Ceyda,42,Memur,Kayseri,4200,kötü
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü


# StringIndexer

In [6]:
from pyspark.ml.feature import StringIndexer

In [7]:
meslek_indexer = StringIndexer() \
.setInputCol("meslek") \
.setOutputCol("meslek_index")

In [8]:
meslek_indexer_model = meslek_indexer.fit(df1)
meslek_indexer_df = meslek_indexer_model.transform(df1)

In [9]:
meslek_indexer_df.toPandas().head(15)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik durum,meslek_index
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0
5,6,Ali,33,Memur,Ankara,4250,kötü,0.0
6,7,Dilek,29,Pazarlamaci,Istanbul,7300,iyi,2.0
7,8,Murat,31,Müzisyen,Istanbul,12000,iyi,1.0
8,9,Ahmet,33,Doktor,Ankara,18000,iyi,3.0
9,10,Muhittin,46,Berber,Istanbul,12000,iyi,4.0


In [10]:
meslek_indexer_df.groupBy(col("meslek")) \
.agg(count("*").alias("sayi")) \
.sort(desc("sayi")) \
.toPandas().head(10)

,meslek,sayi
0,Memur,3
1,Müzisyen,3
2,Pazarlamaci,3
3,Doktor,2
4,Isci,1
5,Tuhafiyeci,1
6,Berber,1
7,Tornacı,1


In [11]:
df2 = meslek_indexer_df

In [12]:
df2.toPandas().head(10)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik durum,meslek_index
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0
5,6,Ali,33,Memur,Ankara,4250,kötü,0.0
6,7,Dilek,29,Pazarlamaci,Istanbul,7300,iyi,2.0
7,8,Murat,31,Müzisyen,Istanbul,12000,iyi,1.0
8,9,Ahmet,33,Doktor,Ankara,18000,iyi,3.0
9,10,Muhittin,46,Berber,Istanbul,12000,iyi,4.0


In [13]:
sehir_indexer = StringIndexer() \
.setInputCol("sehir") \
.setOutputCol("sehir_index")

In [14]:
sehir_indexer_model = sehir_indexer.fit(df2)
sehir_indexer_model.df2 = sehir_indexer_model.transform(df2)
df3 = sehir_indexer_model.df2 

In [15]:
sehir_indexer_model.df2.toPandas().head(20)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik durum,meslek_index,sehir_index
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0,0.0
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,3.0
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0,1.0
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0,0.0
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0,2.0
5,6,Ali,33,Memur,Ankara,4250,kötü,0.0,0.0
6,7,Dilek,29,Pazarlamaci,Istanbul,7300,iyi,2.0,1.0
7,8,Murat,31,Müzisyen,Istanbul,12000,iyi,1.0,1.0
8,9,Ahmet,33,Doktor,Ankara,18000,iyi,3.0,0.0
9,10,Muhittin,46,Berber,Istanbul,12000,iyi,4.0,1.0


In [16]:
df1.groupBy(col("sehir")) \
.agg(count("*").alias("sayi")) \
.sort(desc("sayi")) \
.toPandas().head(20)

,sehir,sayi
0,Ankara,7
1,Istanbul,4
2,Çorum,1
3,Kayseri,1
4,Bursa,1
5,İzmir,1


# OneHotEncoderEstimator AŞAMASI

In [17]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [18]:
encoder = OneHotEncoder() \
.setInputCols(["meslek_index","sehir_index"]) \
.setOutputCols(["meslek_encoded","sehir_encoded"])

In [19]:
encoder_model = encoder.fit(df3)
df4 = encoder_model.transform(df3) 

In [20]:
df4.toPandas().head(20)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,3.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)"
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)"
5,6,Ali,33,Memur,Ankara,4250,kötü,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
6,7,Dilek,29,Pazarlamaci,Istanbul,7300,iyi,2.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
7,8,Murat,31,Müzisyen,Istanbul,12000,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
8,9,Ahmet,33,Doktor,Ankara,18000,iyi,3.0,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
9,10,Muhittin,46,Berber,Istanbul,12000,iyi,4.0,1.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"


# VectorAssembler AŞAMASI

In [21]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

In [22]:
assembler = VectorAssembler() \
.setInputCols(["yas","aylik_gelir","meslek_encoded","sehir_encoded"]) \
.setOutputCol("vectorized_features")

In [23]:
assembler_df = assembler.transform(df4)

In [24]:
import pandas as pd
pd.set_option("display.max_colwidth", 1)

In [25]:
assembler_df.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded,vectorized_features
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,3.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)"
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
3,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)","(23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)"


In [26]:
assembler_df.select("vectorized_features").toPandas().head()

,vectorized_features
0,"(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
1,"(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)"
2,"(30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
3,"(29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
4,"(23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)"


# LabelIndexer Aşaması

In [27]:
label_indexer = StringIndexer() \
.setInputCol("ekonomik durum") \
.setOutputCol("label")

In [28]:
label_indexer_model2 = label_indexer.fit(assembler_df)
label_indexer_df = label_indexer_model2.transform(assembler_df)

In [29]:
label_indexer_df.select("vectorized_features","label").toPandas().head()

,vectorized_features,label
0,"(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
1,"(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)",0.0
2,"(30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
3,"(29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
4,"(23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0


# StandardScaler AŞAMASI

In [32]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler

In [38]:
import math
math.sqrt(math.pow((35-33),2) + math.pow((18000-3500),2))

14500.000137931034

In [39]:
math.sqrt(math.pow((35-33),2))

2.0

In [41]:
math.sqrt(math.pow((18000-3500),2))

14500.0

Şimdi burda standard scaler apmamızın sebebi şu, yukarıda maaşları 18000 ve 3500 olan
2 kişinin maaşlarını standardize etmezsek bir feature 14500 gibi bir katsayı erirsen diğeri sadece 2 veriyor
yani büyük değere sahip olan feature diğerini emiyor ve bu yüzden yenilen feature etkisiz kalıyor ve buda
makine öğrenmesi sonuçlarını olumsuz etkiliyor. Bunun önüne geçmek için standardize ediyoruz ve standardize
ettiğimizde 1 nolu featuresin ammaş katsayısı 0.772... iken 8 nolu kullanıcının maaş kaysayısı 3.97... oldu
fark baya azaldı 14500 nere 2 nere bu yüzden standatdize ettik

In [42]:
scaler = StandardScaler() \
.setInputCol("vectorized_features") \
.setOutputCol("features")

In [43]:
scaler_model = scaler.fit(label_indexer_df)

In [46]:
scaler_df = scaler_model.transform(label_indexer_df)

In [47]:
scaler_df.select("features").toPandas().head(15)

,features
0,"(5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
1,"(6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0)"
2,"(4.292812263480104, 1.9859783574511787, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
3,"(4.149718521364101, 0.9267899001438834, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
4,"(3.29115606866808, 1.0591884573072952, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0)"
5,"(4.722093489828115, 0.9378231132408343, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
6,"(4.149718521364101, 1.610849112154845, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
7,"(4.435906005596108, 2.6479711432682382, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
8,"(4.722093489828115, 3.9719567149023574, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
9,"(6.58231213733616, 2.6479711432682382, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"


# Train - Test Ayırma Aşaması

In [48]:
train_df, test_df = scaler_df.randomSplit([0.8, 0.2], seed = 142)

In [50]:
train_df.toPandas().head(15)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded,vectorized_features,label,features
0,1,Cemal,35,Isci,Ankara,3500,kötü,5.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
1,2,Ceyda,42,Memur,Kayseri,4200,kötü,0.0,3.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)",0.0,"(6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0)"
2,3,Timur,30,Müzisyen,Istanbul,9000,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(30.0, 9000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.292812263480104, 1.9859783574511787, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
3,5,Yasemin,23,Pazarlamaci,Bursa,4800,kötü,2.0,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)","(23.0, 4800.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0,"(3.29115606866808, 1.0591884573072952, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0)"
4,6,Ali,33,Memur,Ankara,4250,kötü,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(4.722093489828115, 0.9378231132408343, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
5,7,Dilek,29,Pazarlamaci,Istanbul,7300,iyi,2.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(29.0, 7300.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.149718521364101, 1.610849112154845, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
6,8,Murat,31,Müzisyen,Istanbul,12000,iyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(31.0, 12000.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.435906005596108, 2.6479711432682382, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
7,10,Muhittin,46,Berber,Istanbul,12000,iyi,4.0,1.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(6.58231213733616, 2.6479711432682382, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
8,12,Harun,43,Tornacı,Ankara,4200,kötü,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(43.0, 4200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(6.15303091098815, 0.9267899001438834, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
9,13,Hakkı,33,Memur,Çorum,3750,kötü,0.0,4.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 1.0)","(33.0, 3750.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)",0.0,"(4.722093489828115, 0.8274909822713244, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175)"


In [51]:
test_df.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded,vectorized_features,label,features
0,4,Burcu,29,Pazarlamaci,Ankara,4200,kötü,2.0,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(29.0, 4200.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(4.149718521364101, 0.9267899001438834, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
1,9,Ahmet,33,Doktor,Ankara,18000,iyi,3.0,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",1.0,"(4.722093489828115, 3.9719567149023574, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
2,11,Hicaziye,47,Tuhafiyeci,Ankara,4800,kötü,7.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(47.0, 4800.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(6.725405879452164, 1.0591884573072952, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
3,15,Şehmuz,41,Müzisyen,Ankara,8700,iyi,1.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(41.0, 8700.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",1.0,"(5.866843426756143, 1.9197790788694726, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"


# Basit Bir Makine Öğrenmesi Modeli

In [52]:
from pyspark.ml.classification import LogisticRegression

In [53]:
lr_object = LogisticRegression() \
.setFeaturesCol("features") \
.setLabelCol("label") \
.setPredictionCol("prediction")

In [54]:
lr_model = lr_object.fit(train_df)

In [55]:
sonuc_df = lr_model.transform(test_df)

In [58]:
sonuc_df.select("label","prediction").toPandas().head(20)

,label,prediction
0,0.0,0.0
1,1.0,1.0
2,0.0,0.0
3,1.0,0.0
